In [9]:
!pip install langchain
!pip install langchain_experimental

In [1]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain_core.messages import SystemMessage

from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain_experimental.chat_models import Llama2Chat
from os.path import expanduser

from langchain_community.llms import LlamaCpp
import re

def lang_chains():
    template_message1 = [
        SystemMessage(content="You are the narrator of a dungeons and dragon game and you assist the player play the game based on the players responses, you ll ask the player to select suggested options answer"),
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
    prompt_template1 = ChatPromptTemplate.from_messages(template_message1)
    template_message2 = [
        SystemMessage(content="You are the narrator of a dungeons and dragon game and assist in generating a text prompt for an image generation model for the given scenario"),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
    prompt_template2 = ChatPromptTemplate.from_messages(template_message2)
    model_path1 = "/Users/naveenrajg/Downloads/llama-2-7b-chat.Q2_K.gguf"
    model_path2 = "/Users/naveenrajg/Downloads/llama-2-7b-chat.Q2_K.gguf"

    llm1 = LlamaCpp(
        model_path=model_path1,
        streaming=False,
        n_ctx=2048,
    )
    llm2 = LlamaCpp(
        model_path=model_path2,
        streaming=False,
        n_ctx=512,
    )
    model1 = Llama2Chat(llm=llm1)
    model2 = Llama2Chat(llm=llm2)
    memory1 = ConversationBufferMemory(memory_key="chat_history", k=2, return_messages=True)
    memory2 = ConversationBufferMemory(memory_key="prompt_history", k=2, return_messages=True)
    chain1 = LLMChain(llm=model1, prompt=prompt_template1, memory=memory1)
    chain2 = LLMChain(llm=model2, prompt=prompt_template2, memory=memory2)
    return chain1,chain2

In [2]:
def add_prefix(text):
    return text + " . create a sentence prompt to generate an image for this scene in format {prompt:'prompt'}"

def extract_prompt(text):
    """
    Extracts the prompt from the given text.
    
    Args:
        text (str): The input text containing the prompt.
        
    Returns:
        str: The extracted prompt, or None if no prompt is found.
    """
    # Define a regular expression pattern to match the prompt
    pattern = r'"(.*?)"'
    
    # Search for the pattern in the text
    match = re.search(pattern, text)
    
    if match:
        # If a match is found, return the prompt (group 1 of the match)
        return match.group(1)
    else:
        # If no match is found, return None
        return None



In [3]:
def LLM_chat(text):
    image_prompt = chain2.run(
        text=add_prefix(text)
    )

    story = chain1.run(
            text=text
        )
    return image_prompt,story

In [4]:
add_prefix('A hungry Knight enters the realm of dragons')

"A hungry Knight enters the realm of dragons . create a sentence prompt to generate an image for this scene in format {prompt:'prompt'}"

In [5]:
chain1,chain2 = lang_chains()

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /Users/naveenrajg/Downloads/llama-2-7b-chat.Q2_K.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head

In [6]:
image_prompt,story = LLM_chat("A hungry Knight enters the realm of dragons")
print(extract_prompt(image_prompt))
print(story)


/Users/naveenrajg/Desktop/Doc/Encode/stab/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(

llama_print_timings:        load time =    3344.93 ms
llama_print_timings:      sample time =      25.06 ms /   236 runs   (    0.11 ms per token,  9415.90 tokens per second)
llama_print_timings: prompt eval time =    8523.89 ms /    84 tokens (  101.47 ms per token,     9.85 tokens per second)
llama_print_timings:        eval time =   17181.48 ms /   235 runs   (   73.11 ms per token,    13.68 tokens per second)
llama_print_timings:       total time =   26099.85 ms /   319 tokens

llama_print_timings:        load time =     991.09 ms
llama_print_timings:      sample time =      14.56 ms /   192 runs   (    0.08 ms per token, 13182.29 tokens per second)
llama_print_timings: prompt eval time =    4895.45 ms /    71 tokens (   68

A brave knight rides his loyal steed into the mystical realm of dragons, his armor gleaming in the fading sunlight as he prepares to face the fire-breathing beasts that lurk within.
  Ah, a brave adventurer has entered the realm of dragons! As their trusty narrator, I will guide them through this fantastical world filled with mythical creatures.
The Knight approaches a bend in the path, and as they round it, they see a magnificent dragon standing in their path. The dragon's scales glint in the sunlight, casting a warm glow over the landscape.
To continue the adventure, you have the following options:
1. Approach the dragon cautiously, ready for any unexpected movements.
2. Hail the dragon and attempt to strike up a conversation.
3. Attempt to sneak past the dragon, hoping to go unnoticed.
4. Charge towards the dragon with weapon drawn, ready to take it down.
Please select one of these options by typing the corresponding number.


In [6]:
image_prompt,story = LLM_chat("Take a short detour through the bushes to avoid any potential danger that may be lurking in the underbrush. This route is more indirect, but it may be safer in the long run.")
print(extract_prompt(image_prompt))
print(story)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    2608.11 ms
llama_print_timings:      sample time =      11.86 ms /   153 runs   (    0.08 ms per token, 12901.59 tokens per second)
llama_print_timings: prompt eval time =    4272.12 ms /    68 tokens (   62.83 ms per token,    15.92 tokens per second)
llama_print_timings:        eval time =   10087.66 ms /   152 runs   (   66.37 ms per token,    15.07 tokens per second)
llama_print_timings:       total time =   14604.49 ms /   220 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     679.95 ms
llama_print_timings:      sample time =      17.40 ms /   230 runs   (    0.08 ms per token, 13219.15 tokens per second)
llama_print_timings: prompt eval time =   11588.87 ms /   187 tokens (   61.97 ms per token,    16.14 tokens per second)
llama_print_timings:        eval time =   15989.82 ms /   229 runs   (   69.82 ms per token,    14.32 tokens per second)
llama_print_timings:       to

A narrow path winds through the dense underbrush, offering a shadowy refuge from the scorching sun. The air is thick with the pungent scent of wet earth and decaying vegetation, and the rustling of small creatures can be heard in the darkness. As you make your way deeper into the thicket, the trees grow taller and closer together, their branches tangling overhead like grasping fingers. The light grows dimmer, and the shadows seem to move and twist, as if alive.
  Ah, a wise choice, my brave adventurer! *chuckles* Taking a detour through the bushes will indeed increase your chances of avoiding any potential danger lurking in the underbrush. *winks*
As you make your way through the dense foliage, the sounds of rustling leaves and snapping twigs fill the air. You can't help but feel a sense of unease as you navigate through the narrow path. The bushes seem to stretch on forever, blocking any clear path to your destination.
Just as you're starting to lose hope, you hear a faint rumbling in